In [ ]:
import pandas as pd
import numpy as np
import pycountry
from projects.ufo_sightings.mini_project_SQL.setup.helper.country_mapping import country_mapping 


# Country table

In [120]:
population = pd.read_csv(r"C:\Users\Martijn\Downloads\population_date.csv")

In [121]:
population[population["Country Name"].str.contains("hong", case=False, na=False)]
population["Country Name"] = population["Country Name"].replace({
    "Slovak Republic": "Slovakia",
    "Turkiye": "Türkiye",
    "Hong Kong SAR, China": "Hong Kong"
})


In [122]:
# Get a list of all countries
countries_short = [country.alpha_2 for country in pycountry.countries]
countries_name = [country.name for country in pycountry.countries]

countries_df = pd.DataFrame({'handle': countries_short, 'name': countries_name}).reset_index()
countries_df = countries_df.rename(columns={"index":"country_id"})

In [123]:
combined = countries_df.merge(population, how="left", left_on="name", right_on="Country Name")
combined.drop(["Country Name", "Country Code"], axis=1, inplace=True)
combined.rename(columns={"Population":"population"}, inplace=True)

# Movies Dataset cleaning

In [124]:
movies = pd.read_csv(r"C:\Users\Martijn\Downloads\netflix_titles_anandshaw.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [125]:
# How many rows with including the words "alien" and "ufo" in the title or description? 
filtered_df = movies[movies['title'].str.contains('alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', case=False, na=False) | 
                     movies['description'].str.contains('alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', case=False, na=False)]
print(len(filtered_df))

# create new column that indicates if movie has ufo theme
movies['ufo_theme'] = np.where(
    movies['description'].str.contains(
        'alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', 
        case=False, 
        na=False
    ), 
    'yes', 
    'no'
)

movies.columns

174


Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'ufo_theme'],
      dtype='object')

In [126]:
movies['date_added'] = pd.to_datetime(movies['date_added'], errors='coerce')
# Drop rows where 'date_added' is NaT due to coercion errors
movies = movies.dropna(subset=['date_added'])
movies = movies[['date_added', 'ufo_theme', 'release_year', 'type', 'title']]
movies['date_added_formatted'] = movies['date_added'].dt.strftime('%m/%Y')

# UFO reports cleaning

In [204]:
ufo_report  = pd.read_csv(r"C:\Users\Martijn\Downloads\nuforc_reports (1).csv")

In [205]:
ufo_report.info()

# rename columns to lower case
ufo_report.columns = ufo_report.columns.str.lower()

# remove columns 'Date', 'Posted', 'Shape', 'Duration', 'Image', 'Link', 'Summary', 'Text' (to avoid confusion about correct date column and other columns)
ufo_report = ufo_report.drop(columns=['date','posted', 'shape', 'duration', 'image', 'link', 'summary', 'text'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143940 entries, 0 to 143939
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Date_Table  143940 non-null  object
 1   Date        143940 non-null  object
 2   Posted      143940 non-null  object
 3   City        143933 non-null  object
 4   State       143895 non-null  object
 5   Country     143938 non-null  object
 6   Shape       143940 non-null  object
 7   Duration    143915 non-null  object
 8   Image       143940 non-null  object
 9   Link        143940 non-null  object
 10  Summary     143940 non-null  object
 11  Text        143906 non-null  object
dtypes: object(12)
memory usage: 13.2+ MB


In [206]:
# rename column date_table
ufo_report = ufo_report.rename(columns={'date_table': 'date'})
print(ufo_report)


                             date                city    state  country
0                         04/2023            Honolulu       HI      USA
1                         04/2023         Bakersfield  Unknown      USA
2                         04/2023         Castle Dale  Unknown  Unknown
3                         04/2023           Baltimore       MD      USA
4                         03/2023             Madison       WI      USA
...                           ...                 ...      ...      ...
143935                    07/1947             Roswell       NM      USA
143936                    06/1947      Corpus Christi       TX      USA
143937                    06/1952              Auburn       WA      USA
143938  UNSPECIFIED / APPROXIMATE           Troutdale       OR      USA
143939                    06/1950  Budapest (Hungary)  Unknown  Hungary

[143940 rows x 4 columns]


In [207]:
correct_format = r'^\d{1,2}/\d{4}$'

# count rows that do not have date format of MM/YYYY in 'Date_Table'
invalid_rows_count = ufo_report[~ufo_report['date'].str.match(correct_format, na=False)].shape[0]

print(f"invalid rows: {invalid_rows_count}")

# remove rows with invalid date format 
ufo_report = ufo_report[ufo_report['date'].str.match(correct_format, na=False)]
print(ufo_report)

invalid rows: 376
           date                city    state  country
0       04/2023            Honolulu       HI      USA
1       04/2023         Bakersfield  Unknown      USA
2       04/2023         Castle Dale  Unknown  Unknown
3       04/2023           Baltimore       MD      USA
4       03/2023             Madison       WI      USA
...         ...                 ...      ...      ...
143934  07/1954             Oakdale       NY      USA
143935  07/1947             Roswell       NM      USA
143936  06/1947      Corpus Christi       TX      USA
143937  06/1952              Auburn       WA      USA
143939  06/1950  Budapest (Hungary)  Unknown  Hungary

[143564 rows x 4 columns]


In [208]:
# based on unique values from country we clean country column and remove invalid values
ufo_report["country"] = ufo_report["country"].replace(country_mapping)
unique_countries = ufo_report['country'].unique()
print(unique_countries)

# List of invalid values to remove
invalid_countries = ['no', 'unknown', "Unknown" 'none', 'not applicable', 'unknown/at sea', 'unavailable', 'in orbit', 'space', 'atlantic ocean', 'caribbean sea', 'pacific ocean', 'international space station', 'moon', 'mars', 'none', 'not found']

# Convert the 'Country' column to lowercase and filter out invalid values
ufo_report = ufo_report[~ufo_report['country'].str.lower().isin(invalid_countries)]

['United States' 'Unknown' 'United Kingdom' 'Canada' 'Turkey' 'India'
 'Australia' 'Malta' 'Switzerland' 'France' 'Guam' 'Puerto Rico' 'Ukraine'
 'Mexico' 'Ireland' 'Japan' 'Germany' 'Poland' 'New Zealand' 'Pakistan'
 'South Africa' 'Papua New Guinea' 'Kenya' 'Thailand' 'Israel' 'Denmark'
 'Malaysia' 'Lebanon' 'Argentina' 'Myanmar' 'Cambodia' 'Croatia' 'Brazil'
 'North Macedonia' 'Cyprus' 'China' 'Panama' 'Romania' 'In Orbit'
 'Trinidad and Tobago' 'Jamaica' 'Luxembourg' 'Italy' 'Bulgaria' 'Iran'
 'Spain' 'Portugal' 'Indonesia' 'South Korea' 'Belgium' 'Netherlands'
 'Philippines' 'Guatemala' 'Sri Lanka' 'Jordan' 'Afghanistan' 'Finland'
 'Taiwan' 'Algeria' 'Venezuela' 'Bahamas'
 'South Georgia and the South Sandwich Islands' 'Costa Rica' 'Honduras'
 'Lithuania' 'Bahrain' 'Palau' 'Mozambique' 'Dominican Republic' 'Belize'
 'Slovenia' 'Bosnia and Herzegovina' 'Colombia' 'Greece' 'Hungary'
 'Latvia' 'Sweden' 'Portereco  2 miles of the coast' 'Peru' 'Armenia'
 'Mauritius' 'Egypt' 'Nigeria' 

In [209]:
# create Country_Code column
# Funktion zum Abrufen des ISO 3166-1 Alpha-2 Ländercodes
def get_country_code(country_name):
    if isinstance(country_name, str):  # Prüfen, ob es eine Zeichenkette ist
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2 if country else None
    return None  # Falls der Wert kein String ist (z. B. NaN)

# Beispiel-Datenframe erstellen (falls noch nicht vorhanden)
# ufo_report = pd.read_csv("deine_datei.csv")  # Falls Daten aus einer CSV geladen werden

# Neue Spalte "Country_Code" erstellen
ufo_report['country_code'] = ufo_report['country'].apply(lambda x: get_country_code(x) 
    if isinstance(x, str) and x.lower() not in ['non applicable', 'unknown', 'in orbit', 'at sea'] else None)

# Zeilen mit fehlendem "Country_Code" entfernen
ufo_report = ufo_report.dropna(subset=['country_code'])

In [210]:
ufo_report.sort_values(by="date")

,date,city,state,country,country_code
140400,01/1910,Kirksville (near),MO,United States,US
140426,01/1929,Santa Teresa,NM,United States,US
140493,01/1943,Fiji Islands (S. Pacific Ocean),Unknown,Fiji,FJ
140521,01/1944,Wilderness (near western MD),WV,United States,US
140490,01/1944,San Diego,CA,United States,US
...,...,...,...,...,...
49232,12/2022,Chokoloskee,FL,United States,US
105562,12/2022,Lompoc,CA,United States,US
76494,12/2022,Denver,CO,United States,US
134822,12/2022,Sarasota,FL,United States,US


In [211]:
# Merge ufo_report with countries_df to get the corresponding country_id
ufo_report = ufo_report.merge(countries_df[['handle', 'country_id']], how='left', left_on='country_code', right_on='handle')

# Replace the country_code column with the country_id column
ufo_report['country_code'] = ufo_report['country_id']

# Drop unnecessary columns
ufo_report.drop(columns=['handle', 'country_id', "country"], inplace=True)
ufo_report = ufo_report.rename(columns={"country_code":"country_id"})

# Subscribers Cleaning

In [212]:
subscribers = pd.read_csv(r"C:\Users\Martijn\Downloads\subscribers_netflix_2024 (1).csv")

In [136]:
subscribers = subscribers.rename(columns={'estimated_subscribers': 'subscribers'}).reset_index(drop=True)


In [137]:
# Merge ufo_report with countries_df to get the corresponding country_id
subscribers = subscribers.merge(countries_df[['handle', 'country_id']], how='left', left_on='country_code', right_on='handle')

# Replace the country_code column with the country_id column
subscribers['country_code'] = subscribers['country_id']

# Drop unnecessary columns
subscribers.drop(columns=['handle', 'country_id', "country"], inplace=True)
subscribers = subscribers.rename(columns={"country_code":"country_id"})

In [138]:
# Get unique entries from both columns
unique_dates_ufo = ufo_report["date"].unique()
unique_dates_movies = movies["date_added_formatted"].unique()

# Combine and get all unique entries
all_unique_dates = pd.unique(np.concatenate((unique_dates_ufo, unique_dates_movies)))
# Create a mapping of dates to their indices in all_unique_dates
date_to_index = {date: idx for idx, date in enumerate(all_unique_dates)}

# Replace dates in ufo_report["date"] with their corresponding indices
ufo_report["date_index"] = ufo_report["date"].map(date_to_index)

# Replace dates in movies["date_added_formatted"] with their corresponding indices
movies["date_index"] = movies["date_added_formatted"].map(date_to_index)

In [213]:
ufo_report["date"].sort_values()

139569    01/1910
139588    01/1929
139621    01/1943
139645    01/1944
139618    01/1944
           ...   
49014     12/2022
104976    12/2022
76109     12/2022
134027    12/2022
53492     12/2022
Name: date, Length: 142677, dtype: object

In [202]:
# Extract the year from the 'date' column and filter rows with year >= 1900
ufo_report = ufo_report[ufo_report['date'].str.extract(r'(\d{4})')[0].astype(int) >= 1900]

In [216]:
import os

def get_folder_structure(path):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)  # count the folder depth
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 4 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")

get_folder_structure(r"G:\My Drive\Ironhack\projects\ufo_sightings\mini_project_SQL")

mini_project_SQL/
    README.md
    data_cleaning.ipynb
    database_init.ipynb
    .git/
        description
        packed-refs
        HEAD
        config
        index
        COMMIT_EDITMSG
        hooks/
            applypatch-msg.sample
            commit-msg.sample
            fsmonitor-watchman.sample
            post-update.sample
            pre-applypatch.sample
            pre-commit.sample
            pre-merge-commit.sample
            pre-push.sample
            pre-rebase.sample
            pre-receive.sample
            prepare-commit-msg.sample
            push-to-checkout.sample
            sendemail-validate.sample
            update.sample
        info/
            exclude
        objects/
            pack/
                pack-43996b0c891f029a4389a5080290faaf78d5831c.pack
                pack-43996b0c891f029a4389a5080290faaf78d5831c.idx
                pack-43996b0c891f029a4389a5080290faaf78d5831c.rev
            info/
            1a/
                eca9a105fcd5

In [214]:
ufo_report[ufo_report['date'].str.extract(r'(\d{4})')[0].astype(int) <= 1900]

,date,city,state,country_id
139542,06/1400,Myers Spring Canyon,TX,234
139543,02/1721,Crescent City,CA,234
139544,03/1861,New York City (Manhattan),NY,234
139545,09/1639,Muddy River (Brookline)(Boston),MA,234
139546,05/1864,Cave Spring,GA,234
139547,04/1561,Nurnburg (Germany),Unknown,59
139548,04/1561,Nuremberg (Germany),Unknown,59
139549,06/1898,Unspecified location,OK,234
139552,12/1762,"Lulworth, Dorsetshire (near) (UK/England)",Unknown,79
139553,08/1860,Cherokee,NC,234
